In [1]:
# work in progress

In [2]:
"""
Video classification with Keras and rolling prediction averaging
We are now ready to implement video classification with Keras via rolling prediction accuracy!

To create this script we’ll take advantage of the temporal nature of videos, specifically the assumption that subsequent frames in a video will have similar semantic contents.

By performing rolling prediction accuracy we’ll be able to “smoothen out” the predictions and avoid “prediction flickering”.
"""

'\nVideo classification with Keras and rolling prediction averaging\nWe are now ready to implement video classification with Keras via rolling prediction accuracy!\n\nTo create this script we’ll take advantage of the temporal nature of videos, specifically the assumption that subsequent frames in a video will have similar semantic contents.\n\nBy performing rolling prediction accuracy we’ll be able to “smoothen out” the predictions and avoid “prediction flickering”.\n'

In [3]:
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2
import pandas as pd

C:\Users\Light\anaconda3\envs\dogs-vs-cats\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Light\anaconda3\envs\dogs-vs-cats\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Light\anaconda3\envs\dogs-vs-cats\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Light\anaconda3\envs\dogs-vs-cats\lib\site-

In [4]:
print("[INFO] loading model")
model = load_model("action_detection_tlearn_VGG16_img_aug_cnn_super_com.h5")

[INFO] loading model


In [5]:
# loading test data
# directory where all the data is present
base_dir = "D:\\something-something-project\\data\\"
video_dir = 'D:\\something-something-project\\data\\videos\\20bn-something-something-v2\\'

test_data = pd.read_json(base_dir + 'something-something-v2-test.json')

In [6]:
test_data.head()

,id
0,1420
1,166429
2,53930
3,73548
4,142328


In [9]:
# initialize the video stream, pointer to output video file, and
# frame dimensions
# let's load one video for classification
labels = ['Dropping_something',
          'Holding_something', 
          'Moving_something', 
          'Picking_something', 
          'Poking_something', 
          'Pouring_something', 
          'Putting_something', 
          'Showing_something', 
          'Tearing_something']

vs = cv2.VideoCapture("D:\\something-something-project\\data\\videos\\20bn-something-something-v2\\25020.webm")
writer = None
(W, H) = (None, None)
Q = deque(maxlen=128)
# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        print ("breaking")
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # clone the output frame, then convert it from BGR to RGB
    # ordering, resize the frame to a fixed 224x224, and then
    # perform mean subtraction
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (84, 84)).astype("float32")
    frame /= 255

    # make predictions on the frame and then update the predictions
    # queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    # print (preds)
    Q.append(preds)

    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    # print (results)
    i = np.argmax(results)
    # print (i)
    label = labels[i]
    # print ("###################################")
    # draw the activity on the output frame
    text = "activity: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*'VP90')
        writer = cv2.VideoWriter("./sampleoutput.webm", fourcc, 5, (W, H), True)

    # write the output frame to disk
    writer.write(output)

    # show the output image
    # cv2.imshow("Output", output)
#     key = cv2.waitKey(1) & 0xFF

#     # if the `q` key was pressed, break from the loop
#     if key == ord("q"):
#         break
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

breaking
[INFO] cleaning up...
